In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
print(train)
print(test)

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
corr = train.corr()
corr

In [ ]:
def show_cat_val_counts(data):
    for cat_feature in ["HomePlanet","CryoSleep","Destination","VIP"]:
        cat_cols = data[cat_feature].value_counts()
        print(cat_cols)
show_cat_val_counts(train)

In [ ]:
show_cat_val_counts(test)

In [ ]:
def show_vis_data(data):
    for cat_feature in ["VIP","Destination","CryoSleep","HomePlanet"]:
        for num_feature in ["Age","RoomService","VRDeck","Spa"]:
            sns.catplot(data=data, x="Transported", y=num_feature, hue=cat_feature, kind="boxen") 
show_vis_data(train)

In [ ]:
f, ax = plt.subplots(figsize=(9,9))
sns.heatmap(corr, annot=True, linewidth=5, ax=ax)

In [ ]:
def cat_fill(data):
    for cat_feature in ["HomePlanet","CryoSleep","Destination","VIP"]:
        data[cat_feature].fillna(data[cat_feature].mode()[0], inplace=True)
    print(data.isnull().sum())
cat_fill(train)
cat_fill(test)

In [ ]:
def age_fill_(data):
  age_fill = data.groupby(['HomePlanet', 'Destination'])['Age'].mean()
  data['Age'] = data.apply(lambda x: age_fill[x['HomePlanet'], x['Destination']] if pd.isnull(x['Age']) else x['Age'], axis=1)
age_fill_(train)
age_fill_(test)

In [ ]:
train.isnull().sum()

In [ ]:
train['Cabin']=train['Cabin'].fillna(train["Cabin"].mode()[0])
train['Deck']=train['Cabin'].apply(lambda x : x.split('/')[0])
train["Num"]=train["Cabin"].apply(lambda x : int(x.split('/')[1]))
train['Side']=train['Cabin'].apply(lambda x : x.split('/')[2])

test['Cabin']=test['Cabin'].fillna(train["Cabin"].mode()[0])
test['Deck']=test['Cabin'].apply(lambda x : x.split('/')[0])
test["Num"]=test["Cabin"].apply(lambda x : int(x.split('/')[1]))
test['Side']=test['Cabin'].apply(lambda x : x.split('/')[2])

In [ ]:
def convert_datas(data):
  for feature in ["HomePlanet", "Destination", "Deck", "Side"]:
    new_cols = pd.get_dummies(data[feature])
    data = data.join(new_cols)
    data.drop(feature, axis=1, inplace=True)
  return data
train = convert_datas(train)
test = convert_datas(test)
train.head(10)

In [ ]:
def bool_to_str(data):
    for feature in ["CryoSleep", "VIP"]:
        data[feature] = data[feature].map({False:"0", True:"1"})
    return data
train = bool_to_str(train)
test = bool_to_str(test)
train.head()

In [ ]:
train.head(10)

In [ ]:
train[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]=train[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].fillna(0)
train.isnull().sum()
test[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]=test[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].fillna(0)

In [ ]:
train.head(10)

In [ ]:
train.corr()

In [ ]:
train.drop("Name", inplace=True, axis=1)
train.head(10)
test.drop("Name", inplace=True, axis=1)

In [ ]:
train.isnull().sum()

In [ ]:
train["TotalBill"] = train["RoomService"]+train["FoodCourt"]+train["ShoppingMall"]+train["Spa"]+train["VRDeck"]
train.head(10)
test["TotalBill"] = test["RoomService"]+test["FoodCourt"]+test["ShoppingMall"]+test["Spa"]+test["VRDeck"]

In [ ]:
train.info()

In [ ]:
train=pd.get_dummies(train,columns=['CryoSleep','VIP'],drop_first=True)
test=pd.get_dummies(test,columns=['CryoSleep','VIP'],drop_first=True)

In [ ]:
#train.drop("PassengerId")
train.drop(["PassengerId","Cabin"], inplace=True, axis=1)
test.drop(["PassengerId","Cabin"], inplace=True, axis=1)

In [ ]:
train.head()

In [ ]:
test.info()

In [ ]:
test.info()

In [ ]:
train.info()

In [ ]:
#test["TotalBill"] = test["RoomService"]+test["FoodCourt"]+test["ShoppingMall"]+test["Spa"]+test["VRDeck"]
test.head(10)

In [ ]:
train.head(10)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
X=train.drop('Transported',axis=1)
y=train['Transported']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.12,stratify=y,random_state=42)

In [ ]:
rf = RandomForestClassifier(n_estimators=250,n_jobs=-1)
rf=rf.fit(X_train,y_train)
acc=rf.score(X_train,y_train)
acc

In [ ]:
y_pred=rf.predict(X_test)
final_res=rf.predict(test)
final_res=pd.DataFrame({'Transported':final_res},index=test.index)

In [ ]:
print(final_res)

In [ ]:
print(y_pred)

In [ ]:
dftest = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
toSubmit = dftest[['PassengerId']]
toSubmit['Transported'] = final_res

In [ ]:
toSubmit.to_csv('submission.csv', index=None)